<a href="https://colab.research.google.com/github/akan72/comp790/blob/master/notebooks/convAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
cd comp790

/content/comp790


In [0]:
import random 

import torch
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric.transforms as T

from torch_geometric.nn import (NNConv, GCNConv, GraphConv, TopKPooling, graclus, max_pool, max_pool_x,
                                global_mean_pool)
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import MNISTSuperpixels



In [0]:
train_data = MNISTSuperpixels(root='/tmp/MNIST', train=True, transform=T.Cartesian())
test_data = MNISTSuperpixels(root='/tmp/MNIST', train=False, transform=T.Cartesian())

d = train_data


In [0]:
class GraphAE(nn.Module):
  
  def __init__(self, code_size):
    super().__init__()
    self.code_size = code_size
  
    # Encoder 
#     self.enc_conv1 = GCNConv(1, 20)
#     self.enc_conv2 = GCNConv(10, 20)

    n1 = nn.Sequential(nn.Linear(2, 25), nn.ReLU(), nn.Linear(25, 32))
    n2 = nn.Sequential(nn.Linear(2, 25), nn.ReLU(), nn.Linear(25, 2048))

    self.enc_conv1 = NNConv(d.num_features, 32, n1)
    self.enc_conv2 = NNConv(32, 64, n2)
    self.enc_lin1 = nn.Linear(4 * 4 * 20, 50)
    self.enc_lin2 = nn.Linear(50, self.code_size)

    # Decoder 
    self.dec_lin1 = nn.Linear(self.code_size, 160)
    self.dec_lin2 = nn.Linear(160, IMAGE_SIZE)
    
  def forward(self, data):
    code = self.encode(data)
    out = self.decode(code)
    return out, code
  
  def encode(self, data):
    x, edge_index = data.x, data.edge_index
    
    
    code = self.enc_conv1(data, edge_index, data.edge_attr)
    code = F.selu(F.max_pool2d(code, 2))
    
    code = self.enc_conv2(code, edge_index, data.edge_attr)
    code = F.selu(F.max_pool2d(code, 2))
    
    code = F.selu(self.enc_lin1(code))
    cdoe = self.enc_lin2(code)
    return code
  
  def decode(self, data):
    out = F.selu(self.dec_lin1(code))
    out = F.sigmoid(self.dec_lin2(out))
    return out
    

In [0]:
# Model params
IMAGE_SIZE = 784
IMAGE_WIDTH = IMAGE_HEIGHT = 28

code_size = 20
num_epochs = 20
batch_size = 128

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [137]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphAE(code_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(num_epochs):
  model.train()
  print("Epoch %d " % epoch)
  
  for data in train_loader:
    
    data = data.to(device)
    out, code = model(row)
    
    optimizer.zero_grad()
    loss = F.nll_loss(out, row)
    loss.backward()
    
    optimizer.step()
    #print('Loss = %.3f' % loss.data[0])

    
# def train(epoch):
#   model.train()
#   optimizer.zero_grad()



Epoch 0 


AttributeError: ignored

In [125]:
# Reconstruction

test_image = random.choice(test_data)
print(test_image)
#test_reconstuction, _ = model(test_image)



Data(edge_attr=[1425, 2], edge_index=[2, 1425], pos=[75, 2], x=[75, 1], y=[1])
